## پروژه اول درس هوش مصنوعی با موضوع الگوریتم های جستجوی آگاهانه و ناآگاهانه

### در این پروژه سعی میکنیم با استفاده از الگوریتم های زیر مساله ی یافتن راهی برای جمع آوری معجون ها از معبدی با شکل شطرنجی را حل کنیم

## BFS , IDS & A*

# Imports

In [1]:
#import numpy as np
import time
from copy import deepcopy
import queue as qu

# Body

## Load tests data

In [2]:
tests = []
file = open("test1.in")
tests.append(file.readlines())
file.close()
file = open("test2.in")
tests.append(file.readlines())
file.close()
file = open("test3.in")
tests.append(file.readlines())
file.close()

#### داده های تست ها را از فایل هایی که در پوشه پروژه قرار دارند برای تست برنامه لود می کنیم
#### با توجه به تست های موجود فرض می شود که تعداد سطر ها و ستون ها اعداد تک رقمی می باشند 

## Create tests maps

In [3]:
maps = []
Cs = []
Ks = []
for test in tests:
    num_of_row = int(test[0][0])
    num_of_col = int(test[0][2])
    curr_map = [['e'] * num_of_col for i in range(num_of_row)]
    curr_c = int(test[1][0])
    curr_k = int(test[1][2])
    curr_Cs = []
    curr_Ks = []
    for i in range(0, curr_c):
        curr_Cs.append([int(test[2+i][0]), int(test[2+i][2]), False])
        curr_map[int(test[2+i][0])][int(test[2+i][2])] = 'c'
    for i in range(0, curr_k):
        curr_Ks.append([int(test[2+curr_c+i][0]), int(test[2+curr_c+i][2]), False])
        curr_map[int(test[2+curr_c+i][0])][int(test[2+curr_c+i][2])] = 'k'
    curr_d = int(test[2+curr_c+curr_k])
    for i in range(0, curr_d):
        curr_map[int(test[3+curr_c+curr_k+i][0])][int(test[3+curr_c+curr_k+i][2])] = 'd'
    maps.append(curr_map)
    Cs.append(curr_Cs)
    Ks.append(curr_Ks)


In [4]:
maps

[[['e', 'k', 'c', 'e'],
  ['e', 'e', 'd', 'e'],
  ['c', 'e', 'e', 'e'],
  ['e', 'e', 'e', 'e']],
 [['k', 'e', 'e', 'd'],
  ['e', 'k', 'c', 'd'],
  ['c', 'c', 'e', 'e'],
  ['e', 'k', 'e', 'e']],
 [['e', 'e', 'd', 'e', 'e', 'e'],
  ['e', 'd', 'c', 'e', 'e', 'c'],
  ['e', 'e', 'k', 'e', 'e', 'e'],
  ['d', 'e', 'e', 'e', 'e', 'c'],
  ['e', 'd', 'k', 'c', 'e', 'e'],
  ['e', 'c', 'e', 'e', 'e', 'e']]]

#### مشاهده می شود که نقشه های تست به درستی  لود شده اند

## Model Description

### Actions : Di up, Di down, Di right, Di left (for i in range num of doctors)

### Initial State : A doctor in (0, 0) and the map looks like the test files

### Goal State : Every doctor in (n-1, m-1) and no 'c' remain in the map

### Path cost : sum of moves (Actions) of all doctors (from Initial state to Goal state)

### State : position of elements in the map in a node represents the current state for this node

### Transition Model : اگر دکتری به خانه ای که دارای دارو است وارد شود آن خانه تبدیل به خانه ی خالی شده و یک دکتر در خانه ی پایین سمت چپ جدول ایجاد می شود . اگر دکتری وارد خانه ای دارای معجون شود آن خانه به خانه ی خالی تبدیل می شود . برای حرکت دکتر ها هم تنها در صورتی که خانه ی مورد نظر مجاز باشد و قابل رفتن باشد به آنجا می رود

## Common Functions

In [553]:
def is_in_list(node, ls):
    #result = np.where(expanded == curr_state, True, False)
    #if True in result:
    #    return True
    #return False
    ##############
    for nd in ls:
        if nd.is_same(node):
            return True
    return False
    ############
    #node_list = [node]*len(expanded)
    #result = map(is_same, node_list, expanded)
    #if True in result:
    #    return True
    #return False
    
def eat_drug(Ks, x, y):
    for k in Ks:
        if k[0] == x and k[1] == y:
            if k[2]:
                return False
            else:
                k[2] = True
                return True
            
def pick_confection(Cs, x, y):
    for c in Cs:
        if c[0] == x and c[1] == y:
            c[2] = True
            return
        
def is_dr_in_list(dr, drs, m, n):
    if dr[1] == m-1 and dr[2] == n-1:
        return True
    for d in drs:
        if d[1] == dr[1] and d[2] == dr[2]:
            return True
    return False

## Data Structures

In [554]:
class Fifo:
    def __init__(self):
        self.ls = []
        self.front = 0
        self.rear = 0
        
    def put(self, element):
        self.ls.append(element)
        self.rear += 1
        
    def get(self):
        self.front += 1
        return self.ls[self.front-1]
    
    def empty(self):
        return self.front >= self.rear
        

In [555]:
class Lifo:
    def __init__(self):
        self.ls = []
        self.top = 0
        
    def put(self, element):
        self.ls.append(element)
        self.top += 1
        
    def get(self):
        self.top -= 1
        return self.ls.pop()
    
    def empty(self):
        return self.top <= 0
        

In [570]:
class Node:
    def __init__(self, curr_map, curr_Cs, curr_Ks, path_until_there, path_cost_until_there, curr_Drs, row, col):
        self.map = curr_map
        self.Cs = curr_Cs
        self.Ks = curr_Ks
        self.path = path_until_there
        self.path_cost = path_cost_until_there
        self.Drs = curr_Drs
        self.m = row
        self.n = col
        #self.state = self.calculate_state()
        if self.path_cost == 0:
            if self.map[0][0] == 'k':
                eat_drug(self.Ks, 0, 0)
                self.Drs.append([1, self.m-1, 0])
                
    #def calculate_state(self):
        #string = ""
        #for c in self.Cs:
            #if c[2]:
                #string += '1'
            #else:
                #string += '0'
        #for k in self.Ks:
            #if k[2]:
                #string += '1'
            #else:
                #string += '0'
        #dr_ls = []
        #for dr in self.Drs:
            #dr_ls.append(10 * dr[1] + dr[2])
        #dr_ls.sort()
        #string += str(dr_ls)
        #return string
    
    def Goal_test(self):
        for c in self.Cs:
            if c[2] is False:
                return False
        for dr in self.Drs:
            if dr[1] != self.m-1 or dr[2] != self.n-1:
                return False
        return True
    
    def expand(self, queue, expanded, limit = None):
        ##if expanded:
        ##    if expanded[-1].path_cost < self.path_cost:
        ##        expanded = []
        expanded.append(self)
        if limit != None:
            if self.path_cost == limit:
                return None
        for dr in self.Drs:
            if dr[1] == self.m-1 and dr[2] == self.n-1:
                continue
            if dr[1]+1 < self.m:
                if self.map[dr[1]+1][dr[2]] != 'd':
                    #temp_state = deepcopy(self.state)
                    temp_drs = deepcopy(self.Drs)
                    temp_drs[dr[0]][1] = dr[1]+1
                    temp_Ks = deepcopy(self.Ks)
                    temp_Cs = deepcopy(self.Cs)
                    if self.map[dr[1]+1][dr[2]] == 'k':
                        if eat_drug(temp_Ks, dr[1]+1, dr[2]):
                            temp_drs.append([len(self.Drs), self.m-1, 0])
                    if self.map[dr[1]+1][dr[2]] == 'c':
                        pick_confection(temp_Cs, dr[1]+1, dr[2])
                    #temp_state[dr[1]+1][dr[2]] = 'e'
                    temp_path = deepcopy(self.path)
                    temp_path.append([dr[0], "D"])
                    temp_node = Node(self.map, temp_Cs, temp_Ks, temp_path, self.path_cost+1, temp_drs, self.m, self.n)
                    if not is_in_list(temp_node, expanded):
                        if temp_node.Goal_test():
                            return temp_node
                        if limit != None:
                            queue.put(temp_node)
                        elif not is_in_list(temp_node, queue.ls):
                            queue.put(temp_node)
            if dr[1] > 0:
                if self.map[dr[1]-1][dr[2]] != 'd':
                    #temp_state = deepcopy(self.state)
                    temp_drs = deepcopy(self.Drs)
                    temp_drs[dr[0]][1] = dr[1]-1
                    temp_Ks = deepcopy(self.Ks)
                    temp_Cs = deepcopy(self.Cs)
                    if self.map[dr[1]-1][dr[2]] == 'k':
                        if eat_drug(temp_Ks, dr[1]-1, dr[2]):
                            temp_drs.append([len(self.Drs), self.m-1, 0])
                    if self.map[dr[1]-1][dr[2]] == 'c':
                        pick_confection(temp_Cs, dr[1]-1, dr[2])
                    #temp_state[dr[1]-1][dr[2]] = 'e'
                    temp_path = deepcopy(self.path)
                    temp_path.append([dr[0], "U"])
                    temp_node = Node(self.map, temp_Cs, temp_Ks, temp_path, self.path_cost+1, temp_drs, self.m, self.n)
                    if not is_in_list(temp_node, expanded):
                        if temp_node.Goal_test():
                            return temp_node
                        if limit != None:
                            queue.put(temp_node)
                        elif not is_in_list(temp_node, queue.ls):
                            queue.put(temp_node)
            if dr[2]+1 < self.n:
                if self.map[dr[1]][dr[2]+1] != 'd':
                    #temp_state = deepcopy(self.state)
                    temp_drs = deepcopy(self.Drs)
                    temp_drs[dr[0]][2] = dr[2]+1
                    temp_Ks = deepcopy(self.Ks)
                    temp_Cs = deepcopy(self.Cs)
                    if self.map[dr[1]][dr[2]+1] == 'k':
                        if eat_drug(temp_Ks, dr[1], dr[2]+1):
                            temp_drs.append([len(self.Drs), self.m-1, 0])
                    if self.map[dr[1]][dr[2]+1] == 'c':
                        pick_confection(temp_Cs, dr[1], dr[2]+1)
                    #temp_state[dr[1]][dr[2]+1] = 'e'
                    temp_path = deepcopy(self.path)
                    temp_path.append([dr[0], "R"])
                    temp_node = Node(self.map, temp_Cs, temp_Ks, temp_path, self.path_cost+1, temp_drs, self.m, self.n)
                    if not is_in_list(temp_node, expanded):
                        if temp_node.Goal_test():
                            return temp_node
                        if limit != None:
                            queue.put(temp_node)
                        elif not is_in_list(temp_node, queue.ls):
                            queue.put(temp_node)
            if dr[2] > 0:
                if self.map[dr[1]][dr[2]-1] != 'd':
                    #temp_state = deepcopy(self.state)
                    temp_drs = deepcopy(self.Drs)
                    temp_drs[dr[0]][2] = dr[2]-1
                    temp_Ks = deepcopy(self.Ks)
                    temp_Cs = deepcopy(self.Cs)
                    if self.map[dr[1]][dr[2]-1] == 'k':
                        if eat_drug(temp_Ks, dr[1], dr[2]-1):
                            temp_drs.append([len(self.Drs), self.m-1, 0])
                    if self.map[dr[1]][dr[2]-1] == 'c':
                        pick_confection(temp_Cs, dr[1], dr[2]-1)
                    #temp_state[dr[1]][dr[2]-1] = 'e'
                    temp_path = deepcopy(self.path)
                    temp_path.append([dr[0], "L"])
                    temp_node = Node(self.map, temp_Cs, temp_Ks, temp_path, self.path_cost+1, temp_drs, self.m, self.n)
                    if not is_in_list(temp_node, expanded):
                        if temp_node.Goal_test():
                            return temp_node
                        if limit != None:
                            queue.put(temp_node)
                        elif not is_in_list(temp_node, queue.ls):
                            queue.put(temp_node)
            if limit == None:
                break  
        return None
    
    #Implementation with limited DFS
    #def LDFS_expand(self, lifo, expanded, limit):
     #   expanded.append(self)
       # if self.path_cost == limit:
        #    return None
     ####################################################################   
    def dexpand(self, queue, expanded, limit = None):
        ##if expanded:
        ##    if expanded[-1].path_cost < self.path_cost:
        ##        expanded = []
        if self.Goal_test():
            return self
        expanded.append(self)
        if limit != None:
            if self.path_cost == limit:
                return None
        for dr in self.Drs:
            if dr[1] == self.m-1 and dr[2] == self.n-1:
                continue
            if dr[1]+1 < self.m:
                if self.map[dr[1]+1][dr[2]] != 'd':
                    #temp_state = deepcopy(self.state)
                    temp_drs = deepcopy(self.Drs)
                    temp_drs[dr[0]][1] = dr[1]+1
                    temp_Ks = deepcopy(self.Ks)
                    temp_Cs = deepcopy(self.Cs)
                    if self.map[dr[1]+1][dr[2]] == 'k':
                        if eat_drug(temp_Ks, dr[1]+1, dr[2]):
                            temp_drs.append([len(self.Drs), self.m-1, 0])
                    if self.map[dr[1]+1][dr[2]] == 'c':
                        pick_confection(temp_Cs, dr[1]+1, dr[2])
                    #temp_state[dr[1]+1][dr[2]] = 'e'
                    temp_path = deepcopy(self.path)
                    temp_path.append([dr[0], "D"])
                    temp_node = Node(self.map, temp_Cs, temp_Ks, temp_path, self.path_cost+1, temp_drs, self.m, self.n)
                    if not is_in_list(temp_node, expanded):
                        if limit != None:
                            queue.put(temp_node)
                        elif not is_in_list(temp_node, queue.ls):
                            queue.put(temp_node)
            if dr[1] > 0:
                if self.map[dr[1]-1][dr[2]] != 'd':
                    #temp_state = deepcopy(self.state)
                    temp_drs = deepcopy(self.Drs)
                    temp_drs[dr[0]][1] = dr[1]-1
                    temp_Ks = deepcopy(self.Ks)
                    temp_Cs = deepcopy(self.Cs)
                    if self.map[dr[1]-1][dr[2]] == 'k':
                        if eat_drug(temp_Ks, dr[1]-1, dr[2]):
                            temp_drs.append([len(self.Drs), self.m-1, 0])
                    if self.map[dr[1]-1][dr[2]] == 'c':
                        pick_confection(temp_Cs, dr[1]-1, dr[2])
                    #temp_state[dr[1]-1][dr[2]] = 'e'
                    temp_path = deepcopy(self.path)
                    temp_path.append([dr[0], "U"])
                    temp_node = Node(self.map, temp_Cs, temp_Ks, temp_path, self.path_cost+1, temp_drs, self.m, self.n)
                    if not is_in_list(temp_node, expanded):
                        if limit != None:
                            queue.put(temp_node)
                        elif not is_in_list(temp_node, queue.ls):
                            queue.put(temp_node)
            if dr[2]+1 < self.n:
                if self.map[dr[1]][dr[2]+1] != 'd':
                    #temp_state = deepcopy(self.state)
                    temp_drs = deepcopy(self.Drs)
                    temp_drs[dr[0]][2] = dr[2]+1
                    temp_Ks = deepcopy(self.Ks)
                    temp_Cs = deepcopy(self.Cs)
                    if self.map[dr[1]][dr[2]+1] == 'k':
                        if eat_drug(temp_Ks, dr[1], dr[2]+1):
                            temp_drs.append([len(self.Drs), self.m-1, 0])
                    if self.map[dr[1]][dr[2]+1] == 'c':
                        pick_confection(temp_Cs, dr[1], dr[2]+1)
                    #temp_state[dr[1]][dr[2]+1] = 'e'
                    temp_path = deepcopy(self.path)
                    temp_path.append([dr[0], "R"])
                    temp_node = Node(self.map, temp_Cs, temp_Ks, temp_path, self.path_cost+1, temp_drs, self.m, self.n)
                    if not is_in_list(temp_node, expanded):
                        if limit != None:
                            queue.put(temp_node)
                        elif not is_in_list(temp_node, queue.ls):
                            queue.put(temp_node)
            if dr[2] > 0:
                if self.map[dr[1]][dr[2]-1] != 'd':
                    #temp_state = deepcopy(self.state)
                    temp_drs = deepcopy(self.Drs)
                    temp_drs[dr[0]][2] = dr[2]-1
                    temp_Ks = deepcopy(self.Ks)
                    temp_Cs = deepcopy(self.Cs)
                    if self.map[dr[1]][dr[2]-1] == 'k':
                        if eat_drug(temp_Ks, dr[1], dr[2]-1):
                            temp_drs.append([len(self.Drs), self.m-1, 0])
                    if self.map[dr[1]][dr[2]-1] == 'c':
                        pick_confection(temp_Cs, dr[1], dr[2]-1)
                    #temp_state[dr[1]][dr[2]-1] = 'e'
                    temp_path = deepcopy(self.path)
                    temp_path.append([dr[0], "L"])
                    temp_node = Node(self.map, temp_Cs, temp_Ks, temp_path, self.path_cost+1, temp_drs, self.m, self.n)
                    if not is_in_list(temp_node, expanded):
                        if limit != None:
                            queue.put(temp_node)
                        elif not is_in_list(temp_node, queue.ls):
                            queue.put(temp_node)
            if limit == None:
                break  
        return None
        ####################################################
    def is_same(self, node):
        #if len(self.Drs) == len(node.Drs):
        if self.Cs == node.Cs and self.Ks == node.Ks:
            for dr in self.Drs:
                if not is_dr_in_list(dr, node.Drs, self.m, self.n):
                    return False
            return True
        return False

#### import inspect
src = inspect.getsource(qu)


#### print(src)

## Test Implementation with BFS

### این الگوریتم کامل است و همواره جواب بهینه را به دست می آورد ولی از نظر تعداد استیت های دیده شده و زمان اجرا بهینه نیست 

## My Test

In [571]:
maps_copy = deepcopy(maps)

In [572]:
Cs_copy = deepcopy(Cs)

In [573]:
maps_copy[0]

[['e', 'k', 'c', 'e'],
 ['e', 'e', 'd', 'e'],
 ['c', 'e', 'e', 'e'],
 ['e', 'e', 'e', 'e']]

In [574]:
Cs_copy[0] = [[0, 2, False]]

In [575]:
maps_copy[0][2][0] = 'e'

In [576]:
maps_copy[0]

[['e', 'k', 'c', 'e'],
 ['e', 'e', 'd', 'e'],
 ['e', 'e', 'e', 'e'],
 ['e', 'e', 'e', 'e']]

In [577]:
fifo = Fifo()
expanded = []

In [578]:
first_node = Node(maps_copy[0], Cs_copy[0], Ks[0], [], 0, [[0, 0, 0]], len(maps_copy[0]), len(maps_copy[0][0]))
fifo.put(first_node)
#frontier.append(first_node)
#index = 0

In [579]:
first_time = time.time()
while not fifo.empty():
    curr_node = fifo.get()
    answer = curr_node.expand(fifo, expanded)
    if answer != None:
        print("answer found")
        print("path cost =", answer.path_cost)
        print(answer.path)
        break
last_time = time.time()

answer found
path cost = 9
[[0, 'R'], [0, 'R'], [0, 'R'], [0, 'D'], [0, 'D'], [0, 'D'], [1, 'R'], [1, 'R'], [1, 'R']]


In [580]:
bfs_time = last_time - first_time

In [581]:
bfs_time

0.0026082992553710938

In [582]:
len(expanded)

36

## Test 1

In [583]:
fifo = Fifo()
expanded = []

In [584]:
fifo.put(Node(maps[0], Cs[0], Ks[0], [], 0, [[0, 0, 0]], len(maps[0]), len(maps[0][0])))

In [585]:
first_time = time.time()
while not fifo.empty():
    curr_node = fifo.get()
    answer = curr_node.expand(fifo, expanded)
    if answer != None:
        print("answer found")
        print("path cost =", answer.path_cost)
        print(answer.path)
        break
last_time = time.time()

answer found
path cost = 11
[[0, 'R'], [0, 'R'], [0, 'R'], [0, 'D'], [0, 'D'], [0, 'D'], [1, 'U'], [1, 'R'], [1, 'D'], [1, 'R'], [1, 'R']]


In [586]:
bfs_time = last_time - first_time

In [587]:
bfs_time

0.0227508544921875

In [588]:
len(expanded)

63

## Test 2

In [589]:
fifo = Fifo()
expanded = []

In [590]:
fifo.put(Node(maps[1], Cs[1], Ks[1], [], 0, [[0, 0, 0]], len(maps[1]), len(maps[1][0])))

In [591]:
first_time = time.time()
while not fifo.empty():
    curr_node = fifo.get()
    answer = curr_node.expand(fifo, expanded)
    if answer != None:
        print("answer found")
        print("path cost =", answer.path_cost)
        print(answer.path)
        break
last_time = time.time()

answer found
path cost = 11
[[0, 'R'], [0, 'R'], [0, 'D'], [0, 'D'], [0, 'D'], [0, 'R'], [1, 'U'], [1, 'R'], [1, 'R'], [1, 'D'], [1, 'R']]


In [592]:
bfs_time = last_time - first_time

In [593]:
bfs_time

0.023622512817382812

In [594]:
len(expanded)

157

## Test 3

In [595]:
fifo = Fifo()
expanded = []

In [596]:
fifo.put(Node(maps[2], Cs[2], Ks[2], [], 0, [[0, 0, 0]], len(maps[2]), len(maps[2][0])))

In [597]:
first_time = time.time()
while not fifo.empty():
    curr_node = fifo.get()
    answer = curr_node.expand(fifo, expanded)
    if answer != None:
        print("answer found")
        print("path cost =", answer.path_cost)
        print(answer.path)
        break
last_time = time.time()

answer found
path cost = 19
[[0, 'D'], [0, 'D'], [0, 'R'], [0, 'R'], [0, 'U'], [0, 'R'], [0, 'R'], [0, 'R'], [0, 'D'], [0, 'D'], [0, 'D'], [0, 'D'], [1, 'R'], [1, 'R'], [1, 'R'], [1, 'U'], [1, 'D'], [1, 'R'], [1, 'R']]


In [598]:
bfs_time = last_time - first_time

In [599]:
bfs_time

0.45151567459106445

In [600]:
len(expanded)

1010

## Tese Implementation with IDS

### این الگوریتم کامل است و همواره جواب بهینه را می دهد ولی از نظر تعداد استیت ها و زمان اجرا بهینه نیست 

### زمان اجرای این الگوریتم از بی اف اس نیز بیشتر است چرا که به ازای هر بار که لیمیت افزایش پیدا می کند تمام نود ها با ارتفاع کمتر که در مرحله ی قبل دیده شده اند باز بررسی می شوند

## My Test

In [601]:
maps_copy = deepcopy(maps)

In [602]:
Cs_copy = deepcopy(Cs)

In [603]:
maps_copy[0]

[['e', 'k', 'c', 'e'],
 ['e', 'e', 'd', 'e'],
 ['c', 'e', 'e', 'e'],
 ['e', 'e', 'e', 'e']]

In [604]:
Cs_copy[0] = [[0, 2, False]]

In [605]:
maps_copy[0][2][0] = 'e'

In [606]:
maps_copy[0]

[['e', 'k', 'c', 'e'],
 ['e', 'e', 'd', 'e'],
 ['e', 'e', 'e', 'e'],
 ['e', 'e', 'e', 'e']]

In [607]:
lifo = Lifo()
expanded = []

In [608]:
first_node = Node(maps_copy[0], Cs_copy[0], Ks[0], [], 0, [[0, 0, 0]], len(maps_copy[0]), len(maps_copy[0][0]))
lifo.put(first_node)
#frontier.append(first_node)
#index = 0

In [609]:
first_time = time.time()
while not lifo.empty():
    curr_node = lifo.get()
    answer = curr_node.dexpand(lifo, expanded, 9)
    if answer != None:
        print("answer found")
        print("path cost =", answer.path_cost)
        print(answer.path)
        break
last_time = time.time()

answer found
path cost = 9
[[0, 'R'], [1, 'R'], [1, 'R'], [1, 'R'], [0, 'R'], [0, 'R'], [0, 'D'], [0, 'D'], [0, 'D']]


In [610]:
bfs_time = last_time - first_time

In [611]:
bfs_time

0.0020606517791748047

In [612]:
len(expanded)

27

In [613]:
for e in expanded:
    print(e.path)

[]
[[0, 'R']]
[[0, 'R'], [1, 'R']]
[[0, 'R'], [1, 'R'], [1, 'R']]
[[0, 'R'], [1, 'R'], [1, 'R'], [1, 'R']]
[[0, 'R'], [1, 'R'], [1, 'R'], [1, 'R'], [0, 'L']]
[[0, 'R'], [1, 'R'], [1, 'R'], [1, 'R'], [0, 'L'], [0, 'D']]
[[0, 'R'], [1, 'R'], [1, 'R'], [1, 'R'], [0, 'L'], [0, 'D'], [0, 'R']]
[[0, 'R'], [1, 'R'], [1, 'R'], [1, 'R'], [0, 'L'], [0, 'D'], [0, 'R'], [0, 'D']]
[[0, 'R'], [1, 'R'], [1, 'R'], [1, 'R'], [0, 'L'], [0, 'D'], [0, 'R'], [0, 'D'], [0, 'L']]
[[0, 'R'], [1, 'R'], [1, 'R'], [1, 'R'], [0, 'L'], [0, 'D'], [0, 'R'], [0, 'D'], [0, 'R']]
[[0, 'R'], [1, 'R'], [1, 'R'], [1, 'R'], [0, 'L'], [0, 'D'], [0, 'R'], [0, 'D'], [0, 'D']]
[[0, 'R'], [1, 'R'], [1, 'R'], [1, 'R'], [0, 'L'], [0, 'D'], [0, 'D']]
[[0, 'R'], [1, 'R'], [1, 'R'], [1, 'R'], [0, 'L'], [0, 'D'], [0, 'D'], [0, 'D']]
[[0, 'R'], [1, 'R'], [1, 'R'], [1, 'R'], [0, 'R']]
[[0, 'R'], [1, 'R'], [1, 'R'], [1, 'R'], [0, 'R'], [0, 'L']]
[[0, 'R'], [1, 'R'], [1, 'R'], [1, 'R'], [0, 'R'], [0, 'L'], [0, 'L']]
[[0, 'R'], [1, 'R'], 

## Test 1

In [619]:
first_time = time.time() 
answer = None
all_expanded = []
limit = 1

while answer == None:
    lifo = Lifo()
    all_expanded.extend(expanded)
    expanded = []
    lifo.put(Node(maps[0], Cs[0], Ks[0], [], 0, [[0, 0, 0]], len(maps[0]), len(maps[0][0])))
    while not lifo.empty():
        curr_node = lifo.get()
        answer = curr_node.dexpand(lifo, expanded, limit)
        if answer != None:
            print("answer found")
            print("path cost =", answer.path_cost)
            print(answer.path)
            break
    limit += 1
last_time = time.time()

answer found
path cost = 14
[[0, 'D'], [0, 'R'], [0, 'D'], [0, 'L'], [0, 'R'], [0, 'R'], [0, 'R'], [0, 'U'], [0, 'U'], [0, 'L'], [0, 'R'], [0, 'D'], [0, 'D'], [0, 'D']]


In [620]:
bfs_time = last_time - first_time

In [621]:
bfs_time

0.08157157897949219

In [622]:
len(all_expanded)

1029

In [623]:
for e in all_expanded:
    print(e.path)

[]
[[0, 'R']]
[[0, 'R'], [1, 'R']]
[[0, 'R'], [1, 'R'], [1, 'R']]
[[0, 'R'], [1, 'R'], [1, 'R'], [1, 'R']]
[[0, 'R'], [1, 'R'], [1, 'R'], [1, 'R'], [0, 'L']]
[[0, 'R'], [1, 'R'], [1, 'R'], [1, 'R'], [0, 'L'], [0, 'D']]
[[0, 'R'], [1, 'R'], [1, 'R'], [1, 'R'], [0, 'L'], [0, 'D'], [0, 'R']]
[[0, 'R'], [1, 'R'], [1, 'R'], [1, 'R'], [0, 'L'], [0, 'D'], [0, 'R'], [0, 'D']]
[[0, 'R'], [1, 'R'], [1, 'R'], [1, 'R'], [0, 'L'], [0, 'D'], [0, 'R'], [0, 'D'], [0, 'L']]
[[0, 'R'], [1, 'R'], [1, 'R'], [1, 'R'], [0, 'L'], [0, 'D'], [0, 'R'], [0, 'D'], [0, 'L'], [0, 'R']]
[[0, 'R'], [1, 'R'], [1, 'R'], [1, 'R'], [0, 'L'], [0, 'D'], [0, 'R'], [0, 'D'], [0, 'L'], [0, 'R'], [0, 'R']]
[[0, 'R'], [1, 'R'], [1, 'R'], [1, 'R'], [0, 'L'], [0, 'D'], [0, 'R'], [0, 'D'], [0, 'L'], [0, 'R'], [0, 'R'], [0, 'R']]
[[0, 'R'], [1, 'R'], [1, 'R'], [1, 'R'], [0, 'L'], [0, 'D'], [0, 'R'], [0, 'D'], [0, 'L'], [0, 'R'], [0, 'R'], [0, 'R'], [0, 'U']]
[[0, 'R'], [1, 'R'], [1, 'R'], [1, 'R'], [0, 'L'], [0, 'D'], [0, 'R'], [0,

## Test 2

In [624]:
first_time = time.time() 
answer = None
all_expanded = []
limit = 1

while answer == None:
    lifo = Lifo()
    all_expanded.extend(expanded)
    expanded = []
    lifo.put(Node(maps[1], Cs[1], Ks[1], [], 0, [[0, 0, 0]], len(maps[1]), len(maps[1][0])))
    while not lifo.empty():
        curr_node = lifo.get()
        answer = curr_node.dexpand(lifo, expanded, limit)
        if answer != None:
            print("answer found")
            print("path cost =", answer.path_cost)
            print(answer.path)
            break
    limit += 1
last_time = time.time()

answer found
path cost = 15
[[1, 'U'], [1, 'R'], [1, 'L'], [1, 'U'], [0, 'R'], [1, 'D'], [1, 'R'], [1, 'R'], [1, 'R'], [1, 'D'], [0, 'R'], [0, 'D'], [0, 'D'], [0, 'R'], [0, 'D']]


In [625]:
bfs_time = last_time - first_time

In [626]:
bfs_time

5.894128084182739

In [627]:
len(all_expanded)

10021

## Test 3

In [569]:
first_time = time.time() 
answer = None
all_expanded = []
limit = 1

while answer == None:
    lifo = Lifo()
    all_expanded.extend(expanded)
    expanded = []
    lifo.put(Node(maps[2], Cs[2], Ks[2], [], 0, [[0, 0, 0]], len(maps[2]), len(maps[2][0])))
    while not lifo.empty():
        curr_node = lifo.get()
        answer = curr_node.dexpand(lifo, expanded, limit)
        if answer != None:
            print("answer found")
            print("path cost =", answer.path_cost)
            print(answer.path)
            break
    limit += 1
last_time = time.time()

KeyboardInterrupt: 

In [ ]:
bfs_time = last_time - first_time

In [ ]:
bfs_time

In [ ]:
len(all_expanded)

## Implementation with A* with consistent heuristic

### این الگوریتم کامل .......... و همواره جواب بهینه را .............. ولی از نظر تعداد استیت ها و زمان اجرا بهینه نیست 

In [39]:
maps_copy = deepcopy(maps)

## Implementation with weighted A* with Alfa = 1.3 

### این الگوریتم کامل .......... و همواره جواب بهینه را .............. ولی از نظر تعداد استیت ها و زمان اجرا بهینه نیست 

In [40]:
maps_copy = deepcopy(maps)

## Implementation with weighted A* with Alfa = 1.5

### این الگوریتم کامل .......... و همواره جواب بهینه را .............. ولی از نظر تعداد استیت ها و زمان اجرا بهینه نیست 

In [41]:
maps_copy = deepcopy(maps)